In [47]:
import re
from collections import Counter

class HangmanSolver:
    def __init__(self, dictionary_path="english_words.txt"):
        """Initialize the Hangman solver with a dictionary of words."""
        self.load_dictionary(dictionary_path)
        self.word_length = 0
        self.current_pattern = []
        self.possible_words = []
        self.guessed_letters = set()
        
    def load_dictionary(self, dictionary_path):
        """Load the dictionary from a file."""
        try:
            with open(dictionary_path, 'r') as file:
                self.dictionary = [word.strip().upper() for word in file.readlines()]
        except FileNotFoundError:
            print(f"Dictionary file {dictionary_path} not found. Using a default dictionary.")
            # Small built-in dictionary in case the file is not found
            self.dictionary = ["HANGMAN", "PYTHON", "DICTIONARY", "SOLVER", "GAME", 
                              "COMPUTER", "PROGRAMMING", "ALGORITHM", "CHALLENGE"]
    
    def start_solve(self, word_length):
        """Start solving a new hangman puzzle with a word of given length."""
        self.word_length = word_length
        self.current_pattern = ['_'] * word_length
        self.possible_words = [word for word in self.dictionary if len(word) == word_length]
        self.guessed_letters = set()
        
        if not self.possible_words:
            print(f"No {word_length}-letter words found in the dictionary.")
            return None
            
        next_letter = self.get_best_letter()
        return next_letter
    
    def update_with_feedback(self, letter, positions):
        """Update possible words based on feedback for the guessed letter."""
        # Add the letter to guessed letters
        self.guessed_letters.add(letter)
        
        if positions == [0]:  # Letter not in word
            # Remove words containing this letter
            self.possible_words = [word for word in self.possible_words if letter not in word]
        else:
            # Update pattern with the letter at specified positions
            for pos in positions:
                self.current_pattern[pos-1] = letter
            
            # Filter possible words based on letter positions
            new_possible_words = []
            for word in self.possible_words:
                match = True
                
                # Check if letter appears at all specified positions
                for pos in positions:
                    if word[pos-1] != letter:
                        match = False
                        break
                
                # Check if letter doesn't appear at other positions still marked as '_'
                if match:
                    for i in range(self.word_length):
                        if self.current_pattern[i] == '_' and i+1 not in positions and word[i] == letter:
                            match = False
                            break
                
                # Check if word matches the current pattern
                if match:
                    for i in range(self.word_length):
                        if self.current_pattern[i] != '_' and word[i] != self.current_pattern[i]:
                            match = False
                            break
                
                if match:
                    new_possible_words.append(word)
            
            self.possible_words = new_possible_words
        
        # Get next best letter
        next_letter = self.get_best_letter()
        return next_letter
    
    def get_best_letter(self):
        """Get the best letter to guess next based on frequency analysis."""
        if not self.possible_words:
            return "No matching words found in dictionary."
        
        if '_' not in self.current_pattern:
            return "Word solved: " + ''.join(self.current_pattern)
        
        # Count letter frequencies in possible words
        letter_counts = Counter()
        for word in self.possible_words:
            # Only count unguessed letters
            for char in set(word):
                if char not in self.guessed_letters:
                    letter_counts[char] += 1
        
        # If no new letters found, return a message
        if not letter_counts:
            return "No more valid letters to guess."
        
        # Select the most frequent letter
        best_letter = letter_counts.most_common(1)[0][0]
        
        return best_letter
    
    def get_status(self):
        """Return the current status of the solver."""
        return {
            'pattern': ''.join(self.current_pattern),
            'possible_words_count': len(self.possible_words),
            'possible_words': self.possible_words[:10] if len(self.possible_words) <= 10 else []
        }

def main():
    print("Hangman Solver")
    print("==============")
    
    # Ask for dictionary path
    dict_path = input("Enter path to dictionary file (or press Enter for default): ")
    if not dict_path:
        dict_path = "words_alpha.txt"
    
    solver = HangmanSolver(dict_path)
    
    # Get word length
    word_length = int(input("Enter the length of the word: "))
    
    # Start solving
    next_letter = solver.start_solve(word_length)
    if next_letter is None:
        return
    
    status = solver.get_status()
    print(f"Current pattern: {status['pattern']}")
    print(f"Possible words: {status['possible_words_count']}")
    print(f"Next letter to try: {next_letter}")
    
    # Solving loop
    while '_' in solver.current_pattern:
        positions_input = input(f"Enter positions for letter '{next_letter}' (comma-separated, or 0 if not in word): ")
        
        try:
            positions = [int(pos) for pos in positions_input.split(',') if pos.strip()]
            
            next_letter = solver.update_with_feedback(next_letter, positions)
            status = solver.get_status()
            
            print(f"Current pattern: {status['pattern']}")
            print(f"Possible words: {status['possible_words_count']}")
            
            if status['possible_words_count'] <= 10 and status['possible_words']:
                print(f"Possible words: {', '.join(status['possible_words'])}")
                
            if isinstance(next_letter, str) and next_letter.startswith("Word solved"):
                print(next_letter)
                break
            elif isinstance(next_letter, str) and ("No more" in next_letter or "No matching" in next_letter):
                print(next_letter)
                break
            else:
                print(f"Next letter to try: {next_letter}")
                
        except ValueError:
            print("Please enter valid numbers separated by commas.")

if __name__ == "__main__":
    main()

Hangman Solver
Current pattern: ______
Possible words: 29874
Next letter to try: E
Current pattern: ______
Possible words: 13016
Next letter to try: A
Current pattern: ______
Possible words: 5102
Next letter to try: I
Current pattern: _I____
Possible words: 477
Next letter to try: O
Current pattern: _I____
Possible words: 224
Next letter to try: U
Current pattern: _I____
Possible words: 106
Next letter to try: Y
Current pattern: _I____
Possible words: 31
Next letter to try: T
Current pattern: _I____
Possible words: 2
Possible words: DIRNDL, KIRSCH
Next letter to try: R
Current pattern: _I____
Possible words: 0
No matching words found in dictionary.


In [50]:
import re
from collections import Counter

class HangmanSolver:
    def __init__(self, dictionary_path="words_alpha.txt"):
        """Initialize the Hangman solver with a dictionary of words."""
        self.load_dictionary(dictionary_path)
        self.word_length = 0
        self.current_pattern = []
        self.possible_words = []
        self.guessed_letters = set()
        # English letter frequency - general frequency data for better initial guesses
        self.english_letter_freq = {
            'E': 12.02, 'T': 9.10, 'A': 8.12, 'O': 7.68, 'I': 7.31, 'N': 6.95, 'S': 6.28, 
            'R': 6.02, 'H': 5.92, 'D': 4.32, 'L': 3.98, 'U': 2.88, 'C': 2.71, 'M': 2.61, 
            'F': 2.30, 'Y': 2.11, 'W': 2.09, 'G': 2.03, 'P': 1.82, 'B': 1.49, 'V': 1.11, 
            'K': 0.69, 'X': 0.17, 'Q': 0.11, 'J': 0.10, 'Z': 0.07
        }
        # Common vowels and consonants for balanced guessing
        self.vowels = set(['A', 'E', 'I', 'O', 'U'])
        self.consonants = set([c for c in self.english_letter_freq.keys() if c not in self.vowels])
        
    def load_dictionary(self, dictionary_path):
        """Load the dictionary from a file."""
        try:
            with open(dictionary_path, 'r') as file:
                self.dictionary = [word.strip().upper() for word in file.readlines()]
        except FileNotFoundError:
            print(f"Dictionary file {dictionary_path} not found. Using a small built-in dictionary.")
            # Small built-in dictionary in case the file is not found
            self.dictionary = ["HANGMAN", "PYTHON", "DICTIONARY", "SOLVER", "GAME", 
                              "COMPUTER", "PROGRAMMING", "ALGORITHM", "CHALLENGE",
                              "METHOD", "RHYTHM", "SYMBOL", "QUARTZ", "WAVES", "JINX",
                              "JAZZY", "PUZZLE", "FIZZY", "OXYGEN", "PIXEL", "ZOMBIE",
                              "WHISKY", "JUMBO", "QUEASY", "WACKY", "BOXCAR", "FUNNY"]
    
    def start_solve(self, word_length):
        """Start solving a new hangman puzzle with a word of given length."""
        self.word_length = word_length
        self.current_pattern = ['_'] * word_length
        self.possible_words = [word for word in self.dictionary if len(word) == word_length]
        self.guessed_letters = set()
        
        if not self.possible_words:
            print(f"No {word_length}-letter words found in the dictionary.")
            return None
            
        next_letter = self.get_best_letter()
        return next_letter
    
    def update_with_feedback(self, letter, positions):
        """Update possible words based on feedback for the guessed letter."""
        # Add the letter to guessed letters
        self.guessed_letters.add(letter)
        
        if positions == [0]:  # Letter not in word
            # Remove words containing this letter
            self.possible_words = [word for word in self.possible_words if letter not in word]
        else:
            # Update pattern with the letter at specified positions
            for pos in positions:
                self.current_pattern[pos-1] = letter
            
            # Filter possible words based on letter positions
            new_possible_words = []
            for word in self.possible_words:
                match = True
                
                # Check if letter appears at all specified positions
                for pos in positions:
                    if word[pos-1] != letter:
                        match = False
                        break
                
                # Check if letter doesn't appear at other positions still marked as '_'
                if match:
                    for i in range(self.word_length):
                        if self.current_pattern[i] == '_' and i+1 not in positions and word[i] == letter:
                            match = False
                            break
                
                # Check if word matches the current pattern
                if match:
                    for i in range(self.word_length):
                        if self.current_pattern[i] != '_' and word[i] != self.current_pattern[i]:
                            match = False
                            break
                
                if match:
                    new_possible_words.append(word)
            
            self.possible_words = new_possible_words
        
        # Get next best letter
        next_letter = self.get_best_letter()
        return next_letter
    
    def get_letter_expected_value(self, letter, words):
        """Calculate the expected information gain for guessing a letter."""
        # If already guessed, no value
        if letter in self.guessed_letters:
            return -1
            
        # Count letter occurrences by position
        total_words = len(words)
        if total_words == 0:
            return -1
            
        # Group words by their pattern when this letter is revealed
        pattern_groups = {}
        for word in words:
            # Create pattern showing where this letter would appear
            pattern = ['_'] * self.word_length
            for i, char in enumerate(word):
                if char == letter:
                    pattern[i] = letter
            
            pattern_key = ''.join(pattern)
            if pattern_key not in pattern_groups:
                pattern_groups[pattern_key] = 0
            pattern_groups[pattern_key] += 1
        
        # Calculate entropy (information gain)
        entropy = 0
        for pattern, count in pattern_groups.items():
            probability = count / total_words
            # Each pattern splits the word set - more balanced splits are better
            entropy -= probability * (probability * len(words))
        
        return entropy
    
    def get_best_letter(self):
        """Get the best letter to guess next based on advanced strategies."""
        if not self.possible_words:
            return "No matching words found in dictionary."
        
        if '_' not in self.current_pattern:
            return "Word solved: " + ''.join(self.current_pattern)
        
        # For very first guess or large word sets, use pre-computed frequency
        if len(self.guessed_letters) == 0 or len(self.possible_words) > 1000:
            # Get unguessed letters sorted by English frequency
            candidates = [l for l in self.english_letter_freq.keys() if l not in self.guessed_letters]
            candidates.sort(key=lambda l: self.english_letter_freq[l], reverse=True)
            
            # Prioritize vowels in early guesses if word is long enough
            if len(self.guessed_letters) < 2 and self.word_length >= 5:
                for vowel in ['E', 'A', 'O', 'I', 'U']:
                    if vowel not in self.guessed_letters:
                        return vowel
            
            # Otherwise return top frequency letter
            return candidates[0]
        
        # For smaller sets, use more sophisticated analysis
        letter_values = {}
        
        # Check for vowel/consonant balance
        guessed_vowels = self.guessed_letters.intersection(self.vowels)
        guessed_consonants = self.guessed_letters.intersection(self.consonants)
        
        # If we haven't guessed any vowels, prioritize them
        need_vowels = len(guessed_vowels) == 0 and self.word_length > 3
        
        # Calculate letter frequencies in possible words
        letter_counts = Counter()
        for word in self.possible_words:
            # Count only unguessed letters
            for char in set(word):
                if char not in self.guessed_letters:
                    letter_counts[char] += 1
        
        # Calculate expected information gain for each letter
        for letter, count in letter_counts.items():
            # Base value is how many words contain this letter
            value = count / len(self.possible_words)
            
            # Adjust for vowel priority if needed
            if need_vowels and letter in self.vowels:
                value *= 1.5
                
            # Calculate expected information gain
            info_gain = self.get_letter_expected_value(letter, self.possible_words)
            if info_gain > 0:
                value *= (1 + info_gain)
            
            letter_values[letter] = value
        
        # If no unguessed letters found
        if not letter_values:
            return "No more valid letters to guess."
        
        # Get letter with highest value
        best_letter = max(letter_values.items(), key=lambda x: x[1])[0]
        
        return best_letter
    
    def get_status(self):
        """Return the current status of the solver."""
        return {
            'pattern': ''.join(self.current_pattern),
            'guessed': ', '.join(sorted(self.guessed_letters)),
            'possible_words_count': len(self.possible_words),
            'possible_words': self.possible_words[:10] if len(self.possible_words) <= 10 else []
        }

def main():
    print("Optimized Hangman Solver")
    print("=======================")
    
    # Ask for dictionary path
    dict_path = input("Enter path to dictionary file (or press Enter for default): ")
    if not dict_path:
        dict_path = "words_alpha.txt"
    
    solver = HangmanSolver(dict_path)
    
    # Get word length
    word_length = int(input("Enter the length of the word: "))
    
    # Start solving
    next_letter = solver.start_solve(word_length)
    if next_letter is None:
        return
    
    # Track guesses remaining
    guesses_remaining = 6  # Standard hangman
    
    status = solver.get_status()
    print(f"Current pattern: {status['pattern']}")
    print(f"Possible words: {status['possible_words_count']}")
    print(f"Remaining guesses: {guesses_remaining}")
    print(f"Next letter to try: {next_letter}")
    
    # Solving loop
    while '_' in solver.current_pattern and guesses_remaining > 0:
        positions_input = input(f"Enter positions for letter '{next_letter}' (comma-separated, or 0 if not in word): ")
        
        try:
            positions = [int(pos) for pos in positions_input.split(',') if pos.strip()]
            
            # Update guess count
            if positions == [0]:  # Wrong guess
                guesses_remaining -= 1
                
            next_letter = solver.update_with_feedback(next_letter, positions)
            status = solver.get_status()
            
            print(f"Current pattern: {status['pattern']}")
            print(f"Guessed letters: {status['guessed']}")
            print(f"Possible words: {status['possible_words_count']}")
            print(f"Remaining guesses: {guesses_remaining}")
            
            if status['possible_words_count'] <= 10 and status['possible_words']:
                print(f"Possible words: {', '.join(status['possible_words'])}")
                
            if isinstance(next_letter, str) and next_letter.startswith("Word solved"):
                print(next_letter)
                break
            elif isinstance(next_letter, str) and ("No more" in next_letter or "No matching" in next_letter):
                print(next_letter)
                break
            elif guesses_remaining == 0:
                print("Out of guesses! Game over.")
                if status['possible_words']:
                    print(f"The word might have been one of: {', '.join(status['possible_words'][:5])}")
                break
            else:
                print(f"Next letter to try: {next_letter}")
                
        except ValueError:
            print("Please enter valid numbers separated by commas.")

if __name__ == "__main__":
    main()

Optimized Hangman Solver
Current pattern: _____
Possible words: 15921
Remaining guesses: 6
Next letter to try: E
Current pattern: _____
Guessed letters: E
Possible words: 9190
Remaining guesses: 5
Next letter to try: A
Current pattern: _A_A_
Guessed letters: A, E
Possible words: 301
Remaining guesses: 5
Next letter to try: N
Current pattern: _A_A_
Guessed letters: A, E, N
Possible words: 207
Remaining guesses: 4
Next letter to try: L
Current pattern: _A_A_
Guessed letters: A, E, L, N
Possible words: 139
Remaining guesses: 3
Next letter to try: R
Current pattern: _A_A_
Guessed letters: A, E, L, N, R
Possible words: 84
Remaining guesses: 2
Next letter to try: S
Current pattern: _A_A_
Guessed letters: A, E, L, N, R, S
Possible words: 49
Remaining guesses: 1
Next letter to try: M
Current pattern: _A_A_
Guessed letters: A, E, L, M, N, R, S
Possible words: 34
Remaining guesses: 0
Out of guesses! Game over.


In [59]:
import re
from collections import Counter

class HangmanSolver:
    def __init__(self, dictionary_path="words_alpha.txt"):
        self.load_dictionary(dictionary_path)
        self.word_length = 0
        self.current_pattern = []
        self.possible_words = []
        self.guessed_letters = set()
        self.match_found = False  # Switch from optimal order to frequency analysis

        # Optimal guessing order by word length
        self.optimal_order_map = {
            1: list("AI"),
            2: list("AOEIUMBH"),
            3: list("AEOIUYHBCK"),
            4: list("AEOIUYSBF"),
            5: list("SEAOIUYH"),
            6: list("EAIOUSY"),
            7: list("EIAOUS"),
            8: list("EIAOU"),
            9: list("EIAOU"),
            10: list("EIOAU"),
            11: list("EIOAD"),
            12: list("EIOAF"),
            13: list("IEOA"),
            14: list("IEO"),
            15: list("IEA"),
            16: list("IEH"),
            17: list("IER"),
            18: list("IEA"),
            19: list("IEA"),
            20: list("IE")
        }
        self.optimal_queue = []

    def load_dictionary(self, dictionary_path):
        try:
            with open(dictionary_path, 'r') as file:
                self.dictionary = [word.strip().upper() for word in file.readlines()]
        except FileNotFoundError:
            print(f"Dictionary file {dictionary_path} not found. Using a default dictionary.")
            self.dictionary = ["HANGMAN", "PYTHON", "DICTIONARY", "SOLVER", "GAME", 
                               "COMPUTER", "PROGRAMMING", "ALGORITHM", "CHALLENGE"]

    def start_solve(self, word_length):
        self.word_length = word_length
        self.current_pattern = ['_'] * word_length
        self.possible_words = [word for word in self.dictionary if len(word) == word_length]
        self.guessed_letters = set()
        self.match_found = False
        self.optimal_queue = self.optimal_order_map.get(word_length, list("ETAOINSHRDLU"))  # fallback
        
        if not self.possible_words:
            print(f"No {word_length}-letter words found in the dictionary.")
            return None

        return self.get_best_letter()

    def update_with_feedback(self, letter, positions):
        self.guessed_letters.add(letter)

        if positions == [0]:  # Letter not in word
            self.possible_words = [word for word in self.possible_words if letter not in word]
        else:
            self.match_found = True
            for pos in positions:
                self.current_pattern[pos-1] = letter

            new_possible_words = []
            for word in self.possible_words:
                match = True

                for pos in positions:
                    if word[pos-1] != letter:
                        match = False
                        break

                if match:
                    for i in range(self.word_length):
                        if self.current_pattern[i] == '_' and i+1 not in positions and word[i] == letter:
                            match = False
                            break

                if match:
                    for i in range(self.word_length):
                        if self.current_pattern[i] != '_' and word[i] != self.current_pattern[i]:
                            match = False
                            break

                if match:
                    new_possible_words.append(word)

            self.possible_words = new_possible_words

        return self.get_best_letter()

    def get_best_letter(self):
        if not self.possible_words:
            return "No matching words found in dictionary."

        if '_' not in self.current_pattern:
            return "Word solved: " + ''.join(self.current_pattern)

        if not self.match_found:
            while self.optimal_queue:
                next_letter = self.optimal_queue.pop(0)
                if next_letter not in self.guessed_letters:
                    return next_letter

        # Frequency analysis
        letter_counts = Counter()
        for word in self.possible_words:
            for char in set(word):
                if char not in self.guessed_letters:
                    letter_counts[char] += 1

        if not letter_counts:
            return "No more valid letters to guess."

        best_letter = letter_counts.most_common(1)[0][0]
        return best_letter

    def get_status(self):
        return {
            'pattern': ''.join(self.current_pattern),
            'possible_words_count': len(self.possible_words),
            'possible_words': self.possible_words[:10] if len(self.possible_words) <= 10 else []
        }

def main():
    print("Hangman Solver")
    print("==============")

    dict_path = input("Enter path to dictionary file (or press Enter for default): ")
    if not dict_path:
        dict_path = "words_alpha.txt"

    solver = HangmanSolver(dict_path)
    word_length = int(input("Enter the length of the word: "))

    next_letter = solver.start_solve(word_length)
    if next_letter is None:
        return

    status = solver.get_status()
    print(f"Current pattern: {status['pattern']}")
    print(f"Possible words: {status['possible_words_count']}")
    print(f"Next letter to try: {next_letter}")

    while '_' in solver.current_pattern:
        positions_input = input(f"Enter positions for letter '{next_letter}' (comma-separated, or 0 if not in word): ")

        try:
            positions = [int(pos) for pos in positions_input.split(',') if pos.strip()]
            next_letter = solver.update_with_feedback(next_letter, positions)
            status = solver.get_status()

            print(f"Current pattern: {status['pattern']}")
            print(f"Possible words: {status['possible_words_count']}")
            if status['possible_words_count'] <= 10 and status['possible_words']:
                print(f"Possible words: {', '.join(status['possible_words'])}")

            if isinstance(next_letter, str) and next_letter.startswith("Word solved"):
                print(next_letter)
                break
            elif "No more" in next_letter or "No matching" in next_letter:
                print(next_letter)
                break
            else:
                print(f"Next letter to try: {next_letter}")

        except ValueError:
            print("Please enter valid numbers separated by commas.")

if __name__ == "__main__":
    main()


Hangman Solver


ValueError: invalid literal for int() with base 10: ''